In [58]:
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np

In [26]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [27]:
# Container for our player object. It needs to hold our IDs and the fields we will use for entity resolution
class Player:
    def __init__(self, name, nba_id, bbref_id, fga, rbd, ast):
        self.name = name
        self.nba_id = nba_id
        self.bbref_id = bbref_id
        self.fga = fga
        self.rbd = rbd
        self.ast = ast


In [28]:
# Converts a basketball reference season to a season recognized by stats.nba.com (2019 -> 2018-19)
def convert_bbref_season_to_nba_season(season):
    year = int(season)
    last_year = year - 1
    last_two = season[-2:]
    return "{0}-{1}".format(last_year, last_two)


In [34]:
# Basketball reference contains multiple rows for players who have played on multiple teams.
# we only care about the season total for the player so we must deduplicate the rows (selected Team = TOT)
def deduplicate_traded_players(group):
    if len(group) > 1:
        return group[group["Team"] == "TOT"]
    return group


def check_names_fuzzy_match(row):
    row["name_match"] = fuzz.partial_ratio(row["Player"], row["PLAYER_NAME"]) > 60
    return row


In [35]:
season = "2023"

# Read our basketball_reference data
bbref_data = pd.read_csv("basketball_reference_totals_{}.csv".format(season))
# read out stats.nba.com data
nba_data = pd.read_csv("stats_nba_player_data_{}.csv".format(convert_bbref_season_to_nba_season(season)))
# convert the player id from an int to a string
nba_data["PLAYER_ID"] = nba_data["PLAYER_ID"].astype(str)

# take the player name, id, team and fields we will use for deduplication from bbref data
bbref_base_data = bbref_data[["Player", "id", "Team", "FGA", "Total Rebounds", "Assists"]].groupby(
    by="id").apply(deduplicate_traded_players)

In [37]:
# take the player name, id, and fields we will use for deduplication from stats.nba.com data
nba_base_data = nba_data[["PLAYER_ID", "PLAYER_NAME", "FGA", "REB", "AST"]]

# Perform a full outer join on the two dataframes. This allows us to get all of the exact matches
name_matches = bbref_base_data.merge(nba_base_data,
                                     left_on=["Player", "FGA", "Total Rebounds", "Assists"],
                                     right_on=["PLAYER_NAME", "FGA", "REB", "AST"], how="outer")


In [118]:
# take all of the exact matches and rename the columns, 
#we only care about player name and id from each source
name_matches_ids = name_matches.dropna()
name_matches_ids = name_matches_ids[["Player", "id", "PLAYER_NAME", "PLAYER_ID"]]
name_matches_ids.columns = ["bbref_name", "bbref_id", "nba_name", "nba_id"]



In [40]:
# Take all of the rows from the full outer join that have null values. These are the cases where no match was found.
non_matches = name_matches[name_matches.isnull().any(axis=1)]

# take all of the bbref data from non_matches
bbref_non_matches = non_matches[["Player", "id", "FGA", "Total Rebounds", "Assists"]].dropna()

# take all of the stats.nba data from the non_matches
nba_non_matches = non_matches[["PLAYER_NAME", "PLAYER_ID", "FGA", "REB", "AST"]].dropna()

In [109]:
potential_matches = bbref_non_matches.merge(nba_non_matches,
                                     left_on=["Total Rebounds", "Assists"],
                                     right_on=["REB", "AST"], how="outer")
#possible_matches.apply(check_names_fuzzy_match, axis=1)
clean_potential_matches = potential_matches.dropna()
possible_matches = clean_potential_matches.apply(check_names_fuzzy_match, axis=1)
fuzzy_matches = possible_matches[possible_matches["name_match"]][["Player", "id", "PLAYER_NAME", "PLAYER_ID"]]
print(fuzzy_matches)

                 Player         id          PLAYER_NAME PLAYER_ID
0            OG Anunoby  anunoog01         O.G. Anunoby   1628384
1   Patrick Baldwin Jr.  baldwpa01      Patrick Baldwin   1631116
4         Dāvis Bertāns  bertada01        Davis Bertans    202722
5     Bogdan Bogdanović  bogdabo01    Bogdan Bogdanovic    203992
6      Bojan Bogdanović  bogdabo02     Bojan Bogdanovic    202711
7           John Butler  butlejo01      John Butler Jr.   1631219
8         Vlatko Čančar  cancavl01        Vlatko Cancar   1628427
9        Moussa Diabaté  diabamo01       Moussa Diabate   1631217
10          Luka Dončić  doncilu01          Luka Doncic   1629029
11          Jeff Dowtin  dowtije01      Jeff Dowtin Jr.   1630288
12         Goran Dragić  dragigo01         Goran Dragic    201609
13             Rudy Gay    gayru01             Rudy Gay    200752
14           A.J. Green  greenaj01             AJ Green   1631260
15  Juancho Hernangómez  hernaju01  Juancho Hernangomez   1627823
16    Will

In [102]:
problem_matches = potential_matches[potential_matches.isnull().any(1)]
problem_matches_bbref = problem_matches.iloc[:,:5].dropna().sort_values(by = "Player")
problem_matches_nba = problem_matches.iloc[:,5:].dropna().sort_values(by = "PLAYER_NAME")
print(problem_matches_nba)

         PLAYER_NAME PLAYER_ID  FGA_y    REB    AST
39     Aaron Wiggins   1630598  363.0  211.0   80.0
40      Blake Wesley   1631104  215.0   82.0   98.0
41    Charles Bassey   1629646  135.0  193.0   48.0
42    Doug McDermott    203926  523.0  138.0   91.0
43       Isaiah Roby   1629676  155.0  108.0   38.0
44  Keita Bates-Diop   1628966  465.0  247.0  103.0


In [105]:
problem_matches_bbref.reset_index(drop=True, inplace=True)
problem_matches_nba.reset_index(drop=True, inplace=True)

In [106]:
#if len(problem_matches_bbref) == len(problem_matches_nba):
clean_problem_matches = pd.concat([problem_matches_bbref, problem_matches_nba], axis = 1)
print(clean_problem_matches)

             Player         id  FGA_x  Total Rebounds  Assists       PLAYER_NAME PLAYER_ID  FGA_y    REB    AST
0     Aaron Wiggins  wiggiaa01  363.0           210.0     80.0     Aaron Wiggins   1630598  363.0  211.0   80.0
1      Blake Wesley  weslebl01  215.0            81.0     99.0      Blake Wesley   1631104  215.0   82.0   98.0
2    Charles Bassey  bassech01  135.0           193.0     47.0    Charles Bassey   1629646  135.0  193.0   48.0
3    Doug McDermott  mcderdo01  523.0           139.0     91.0    Doug McDermott    203926  523.0  138.0   91.0
4       Isaiah Roby   robyis01  155.0           107.0     38.0       Isaiah Roby   1629676  155.0  108.0   38.0
5  Keita Bates-Diop  bateske01  465.0           248.0    103.0  Keita Bates-Diop   1628966  465.0  247.0  103.0


In [111]:
non_fuzzy_matches = clean_problem_matches[["Player", "id", "PLAYER_NAME", "PLAYER_ID"]]
print(non_fuzzy_matches)

             Player         id       PLAYER_NAME PLAYER_ID
0     Aaron Wiggins  wiggiaa01     Aaron Wiggins   1630598
1      Blake Wesley  weslebl01      Blake Wesley   1631104
2    Charles Bassey  bassech01    Charles Bassey   1629646
3    Doug McDermott  mcderdo01    Doug McDermott    203926
4       Isaiah Roby   robyis01       Isaiah Roby   1629676
5  Keita Bates-Diop  bateske01  Keita Bates-Diop   1628966


In [131]:
all_fuzzy_matches = pd.concat([non_fuzzy_matches, fuzzy_matches])
print(all_fuzzy_matches)

                 Player         id          PLAYER_NAME PLAYER_ID
0         Aaron Wiggins  wiggiaa01        Aaron Wiggins   1630598
1          Blake Wesley  weslebl01         Blake Wesley   1631104
2        Charles Bassey  bassech01       Charles Bassey   1629646
3        Doug McDermott  mcderdo01       Doug McDermott    203926
4           Isaiah Roby   robyis01          Isaiah Roby   1629676
5      Keita Bates-Diop  bateske01     Keita Bates-Diop   1628966
0            OG Anunoby  anunoog01         O.G. Anunoby   1628384
1   Patrick Baldwin Jr.  baldwpa01      Patrick Baldwin   1631116
4         Dāvis Bertāns  bertada01        Davis Bertans    202722
5     Bogdan Bogdanović  bogdabo01    Bogdan Bogdanovic    203992
6      Bojan Bogdanović  bogdabo02     Bojan Bogdanovic    202711
7           John Butler  butlejo01      John Butler Jr.   1631219
8         Vlatko Čančar  cancavl01        Vlatko Cancar   1628427
9        Moussa Diabaté  diabamo01       Moussa Diabate   1631217
10        

In [114]:
print(name_matches_ids)

           bbref_name   bbref_id          nba_name   nba_id
0    Precious Achiuwa  achiupr01  Precious Achiuwa  1630173
1        Steven Adams  adamsst01      Steven Adams   203500
2         Bam Adebayo  adebaba01       Bam Adebayo  1628389
3        Ochai Agbaji  agbajoc01      Ochai Agbaji  1630534
4        Santi Aldama  aldamsa01      Santi Aldama  1630583
..                ...        ...               ...      ...
534    Thaddeus Young  youngth01    Thaddeus Young   201152
535        Trae Young  youngtr01        Trae Young  1629027
536    Omer Yurtseven  yurtsom01    Omer Yurtseven  1630209
537       Cody Zeller  zelleco01       Cody Zeller   203469
538       Ivica Zubac  zubaciv01       Ivica Zubac  1627826

[500 rows x 4 columns]


In [132]:
all_fuzzy_matches = all_fuzzy_matches.rename(columns={"Player": "bbref_name", "id": "bbref_id", "PLAYER_NAME": "nba_name", "PLAYER_ID": "nba_id"})
all_fuzzy_matches.reset_index(drop=True, inplace=True)
name_matches_ids.reset_index(drop=True, inplace=True)
all_ids = pd.concat([name_matches_ids, all_fuzzy_matches])
print(all_ids)


            bbref_name   bbref_id             nba_name   nba_id
0     Precious Achiuwa  achiupr01     Precious Achiuwa  1630173
1         Steven Adams  adamsst01         Steven Adams   203500
2          Bam Adebayo  adebaba01          Bam Adebayo  1628389
3         Ochai Agbaji  agbajoc01         Ochai Agbaji  1630534
4         Santi Aldama  aldamsa01         Santi Aldama  1630583
..                 ...        ...                  ...      ...
34  Xavier Tillman Sr.  tillmxa01       Xavier Tillman  1630214
35   Jonas Valančiūnas  valanjo01    Jonas Valanciunas   202685
36      Nikola Vučević  vucevni01       Nikola Vucevic   202696
37           John Wall   walljo01            John Wall   202322
38     Robert Williams  williro04  Robert Williams III  1629057

[539 rows x 4 columns]


In [133]:
season = "2022-23"
all_ids.to_csv("deduplicate_nba_ids{0}.csv".format(season), index=False)